<a href="https://colab.research.google.com/github/ThiagoFsm/projetos-alura/blob/main/Institui%C3%A7%C3%A3o_Financeira.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Criar um banco de dados local de clientes para uma instituição financeira.

https://raw.githubusercontent.com/alura-cursos/Pandas/main/clientes_banco.csv

In [61]:
import pandas as pd

In [66]:
url = 'https://raw.githubusercontent.com/alura-cursos/Pandas/main/clientes_banco.csv'

In [63]:
import sqlalchemy

In [64]:
from sqlalchemy import create_engine, MetaData, Table, inspect

In [74]:
dados = pd.read_csv(url)
dados

,ID_Cliente,Idade,Grau_escolaridade,Estado_civil,Tamanho_familia,Categoria_de_renda,Ocupacao,Anos_empregado,Rendimento_anual,Tem_carro,Moradia
0,5008804,32,Ensino superior,União-estável,2,Empregado,Outro,12,427500.0,1,Apartamento alugado
1,5008805,32,Ensino superior,União-estável,2,Empregado,Outro,12,427500.0,1,Apartamento alugado
2,5008806,58,Ensino médio,Casado,2,Empregado,Segurança,3,112500.0,1,Casa/apartamento próprio
3,5008808,52,Ensino médio,Solteiro,1,Associado comercial,Vendas,8,270000.0,0,Casa/apartamento próprio
4,5008809,52,Ensino médio,Solteiro,1,Associado comercial,Vendas,8,270000.0,0,Casa/apartamento próprio
...,...,...,...,...,...,...,...,...,...,...,...
438458,6840104,62,Ensino médio,Divorciado,1,Pensionista,Outro,0,135000.0,0,Casa/apartamento próprio
438459,6840222,43,Ensino médio,Solteiro,1,Empregado,Construção Civil,8,103500.0,0,Casa/apartamento próprio
438460,6841878,22,Ensino superior,Solteiro,1,Associado comercial,Vendas,1,54000.0,0,Mora com os pais
438461,6842765,59,Ensino médio,Casado,2,Pensionista,Outro,0,72000.0,0,Casa/apartamento próprio


Criar o banco de dados local com a biblioteca SQLAlchemy.

In [68]:
engine = create_engine('sqlite:///:memory:')

Escrever os dados do arquivo CSV neste banco de dados local.

In [69]:
dados.to_sql('clientes', engine, index=False)

438463

Verificar se os dados foram inseridos corretamente no banco de dados

In [71]:
pd.read_sql_table('clientes', engine).head()

,ID_Cliente,Idade,Grau_escolaridade,Estado_civil,Tamanho_familia,Categoria_de_renda,Ocupacao,Anos_empregado,Rendimento_anual,Tem_carro,Moradia
0,5008804,32,Ensino superior,União-estável,2,Empregado,Outro,12,427500.0,1,Apartamento alugado
1,5008805,32,Ensino superior,União-estável,2,Empregado,Outro,12,427500.0,1,Apartamento alugado
2,5008806,58,Ensino médio,Casado,2,Empregado,Segurança,3,112500.0,1,Casa/apartamento próprio
3,5008808,52,Ensino médio,Solteiro,1,Associado comercial,Vendas,8,270000.0,0,Casa/apartamento próprio
4,5008809,52,Ensino médio,Solteiro,1,Associado comercial,Vendas,8,270000.0,0,Casa/apartamento próprio


Realizar três atualizações no banco de dados:

1.   Atualizar o registro do cliente de ID 6840104 que teve o rendimento anual alterado para 300000.
2.   Excluir o registro do cliente de ID 5008809, pois essa pessoa não possui mais conta na instituição financeira.
3.   Criar um novo registro de cliente seguindo as especificações abaixo:
  *   ID_Cliente: 6850985
  *   Idade: 33
  *   Grau_escolaridade: Doutorado
  *   Estado_civil: Solteiro
  *   Tamanho_familia: 1
  *   Categoria_de_renda: Empregado
  *   Ocupacao: TI
  *   Anos_empregado: 2
  *   Rendimento_anual: 290000
  *   Tem_carro: 0
  *   Moradia: Casa/apartamento próprio

Atualizando o registro do cliente de ID 6840104

In [72]:
query = text('UPDATE clientes SET Rendimento_anual = 300000 WHERE ID_Cliente = 6840104')
with engine.connect() as conn:
  conn.execute(query)
  conn.commit()

Verificar se a atualização ocorreu corretamente

In [77]:
query = text('SELECT ID_Cliente, Rendimento_anual FROM clientes WHERE ID_Cliente = 6840104')
pd.read_sql(query, engine)

,ID_Cliente,Rendimento_anual
0,6840104,300000.0


Excluindo o registro do cliente de ID 5008809

In [78]:
query = text('DELETE FROM clientes WHERE ID_Cliente = 5008809')
with engine.connect() as conn:
  conn.execute(query)
  conn.commit()

Verificar se a exclusão ocorreu corretamente

In [79]:
query = text('SELECT ID_Cliente FROM clientes')
pd.read_sql(query, engine)

,ID_Cliente
0,5008804
1,5008805
2,5008806
3,5008808
4,5008810
...,...
438457,6840104
438458,6840222
438459,6841878
438460,6842765


Criando um novo registro de cliente ID 6850985

In [80]:
query = text('INSERT INTO Clientes (ID_Cliente, Idade, Grau_escolaridade, Estado_civil, ' \
        'Tamanho_familia, Categoria_de_renda, Ocupacao, Anos_empregado, ' \
        'Rendimento_anual, Tem_carro, Moradia)' \
        'VALUES (6850985, 33, "Doutorado", "Solteiro", 1, "Empregado", "TI", ' \
        '2, 290000, 0, "Casa/apartamento próprio")')
with engine.connect() as conn:
  conn.execute(query)
  conn.commit()

Verificar se a inserção ocorreu corretamente

In [82]:
query = text('SELECT * FROM clientes WHERE ID_Cliente = 6850985')
pd.read_sql(query, engine)

,ID_Cliente,Idade,Grau_escolaridade,Estado_civil,Tamanho_familia,Categoria_de_renda,Ocupacao,Anos_empregado,Rendimento_anual,Tem_carro,Moradia
0,6850985,33,Doutorado,Solteiro,1,Empregado,TI,2,290000.0,0,Casa/apartamento próprio
